In [0]:
!pip install xmltodict

In [0]:
input_dim = 228

from PIL import Image , ImageDraw
import os
import glob
import numpy as np
import xmltodict
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import cv2
from skimage import data
from skimage.color import rgb2gray
import pandas as pd


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
image_dim = 228
images_dir = '/content/gdrive/My Drive/Colab Notebooks/training_images'
output_dir = '/content/gdrive/My Drive/Colab Notebooks/processed_data'

xml_filepaths = glob.glob( os.path.join( images_dir , '*.xml' ) )
jpg_filepaths = glob.glob( os.path.join( images_dir , '*.jpg' ) )

print(xml_filepaths)
print(jpg_filepaths)


['/content/gdrive/My Drive/Colab Notebooks/training_images/frame10.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame0.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame100.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame1.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame102.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame101.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame104.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame105.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame103.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame106.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame109.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame11.xml', '/content/gdrive/My Drive/Colab Notebooks/training_images/frame108.xml', '/content/gdrive/My Drive/Colab Notebooks/training_image

In [0]:
images = []
for imagefile in jpg_filepaths:
    image = Image.open( imagefile ).resize( ( input_dim , input_dim ))
    image = np.asarray( image ) / 255.0
    images.append( image )
    
bboxes = []
classes_raw = []
for xmlfile in xml_filepaths:
#    try:
    x = xmltodict.parse( open( xmlfile , 'rb' ) )
    bndbox = x[ 'annotation' ][ 'object' ][ 'bndbox' ]
    bndbox = np.array([ int(bndbox[ 'xmin' ]) , int(bndbox[ 'ymin' ]) , int(bndbox[ 'xmax' ]) , int(bndbox[ 'ymax' ]) ])
    bndbox2 = [ None ] * 4
    bndbox2[0] = bndbox[0]
    bndbox2[1] = bndbox[1]
    bndbox2[2] = bndbox[2]
    bndbox2[3] = bndbox[3]
    bndbox2 = np.array( bndbox2 ) / input_dim
    bboxes.append( bndbox2 )
    classes_raw.append( x[ 'annotation' ][ 'object' ][ 'name' ] )

In [0]:
boxes = np.array( bboxes ) 
encoder = LabelBinarizer()
#classes_onehot = encoder.fit_transform( classes_raw )
classes_raw_trial=pd.DataFrame(classes_raw)
dummy=pd.get_dummies(classes_raw_trial, columns=[0])
dummy1=dummy.values
classes_onehot=dummy1.astype(int)

In [0]:
Y = np.concatenate( [ boxes , classes_onehot ] , axis=1 )
X = np.array( images )

x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.1 )

input_shape = ( input_dim , input_dim , 3 )
dropout_rate = 0.5
alpha = 0.2
num_classes = 1
pred_vector_length = 4 + num_classes

In [0]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
model_layers = [        
	keras.layers.Conv2D(16, kernel_size=(3, 3), strides=1, input_shape=input_shape),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(16, kernel_size=(3, 3), strides=1 ),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Flatten() , 

    keras.layers.Dense( 1240 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 640 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 480 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 120 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 62 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,


    keras.layers.Dense(pred_vector_length),
    keras.layers.LeakyReLU( alpha=alpha ) ,
]

In [0]:
model = keras.Sequential( model_layers )
model.compile(
	optimizer=keras.optimizers.Adam( lr=0.0001 ),
	loss='mean_squared_error',
    metrics=['accuracy']
)


model.fit( 
    x_train ,
    y_train , 
    validation_data=( x_test , y_test ),
    epochs=2 ,
    batch_size=1) 

model.save( '/content/gdrive/My Drive/Colab Notebooks/pretrained_weights/model.h5')









Train on 300 samples, validate on 34 samples
Epoch 1/2





300/300 [==============================] - 71s 236ms/step - loss: 0.1514 - acc: 0.5300 - val_loss: 0.0721 - val_acc: 0.6176
Epoch 2/2
300/300 [==============================] - 69s 229ms/step - loss: 0.0679 - acc: 0.6300 - val_loss: 0.0619 - val_acc: 0.5294


In [0]:
boxes_original = model.predict( x_test )
for i in range( boxes.shape[0] ):
    b = boxes[ i , 0 : 4 ] * input_dim
    img = x_test[i] * 255
    source_img = Image.fromarray( img.astype( np.uint8 ) , 'RGB' )
    draw = ImageDraw.Draw( source_img )
    draw.rectangle( b , outline ="yellow" )
    source_img.save( '/content/gdrive/My Drive/Colab Notebooks/result/image_{}.png'.format( i + 1 ) , 'png' )

In [0]:
from google.colab.patches import cv2_imshow
def detect(frame):
#    img1 = cv2.imread(r'C:\Users\user\Desktop\ultra.jpg')
    image_re1=cv2.resize(frame, (input_dim , input_dim))
    image1 = np.asarray( image_re1 ) / 255.0
    #images1.append( image1 )
    #X = np.array( images1 )
    #data1 = image1.reshape((image1.shape[0], image1.shape[1],image1.shape[2], 1))
    data1 = image1.reshape((1,image1.shape[0], image1.shape[1],image1.shape[2]))
    
    boxes = model.predict( data1)
    b = boxes[ 0 , 0 : 4 ] * input_dim
    
    img = data1 * 255
    data11 = img.reshape((image1.shape[0], image1.shape[1],-1))
    
    img_cv = data11.astype(np.uint8)
    img_cv1 = cv2.rectangle(img_cv,(int(b[2]), int(b[3])),(int(b[0]),int(b[1])),(0,255,0),2)
    
    cv2_imshow(img_cv)

#    source_img = Image.fromarray( img.astype( np.uint8 ) , 'RGB' )
#    cv2.imshow('image', source_img)
    #source_img.show()
#    draw = ImageDraw.Draw( source_img )
#    new_image_avi=draw.rectangle(b , outline ="red" )
    return img_cv1

cap = cv2.VideoCapture(r'/content/gdrive/My Drive/Colab Notebooks/Vedio/Ultrasound_vedio.MP4')

In [0]:
from google.colab.patches import cv2_imshow
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    canvas=detect(frame)
    # Our operations on the frame come here
    # Display the resulting frame
    cv2_imshow(canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
